# 한글 손글씨 인식 서비스 만들기

In [4]:
import os
import numpy as np
import gradio as gr
from PIL import Image


In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
import requests 
import unicodedata
from io import BytesIO
from PIL import Image

processor = TrOCRProcessor.from_pretrained("ddobokki/ko-trocr") 
model = VisionEncoderDecoderModel.from_pretrained("ddobokki/ko-trocr")
tokenizer = AutoTokenizer.from_pretrained("ddobokki/ko-trocr")

url = "https://raw.githubusercontent.com/ddobokki/ocr_img_example/master/g.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content))

pixel_values = processor(img, return_tensors="pt").pixel_values 
generated_ids = model.generate(pixel_values, max_length=64)
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
generated_text = unicodedata.normalize("NFC", generated_text)
print(generated_text)

/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


지방토목주사


In [6]:
class TrOCRInferencer:
    def __init__(self):
        print("[info] init TrOCR Inferencer")
        self.processor = TrOCRProcessor.from_pretrained("ddobokki/ko-trocr")
        self.model = VisionEncoderDecoderModel.from_pretrained("ddobokki/ko-trocr")
        self.tokenizer = AutoTokenizer.from_pretrained("ddobokki/ko-trocr")
        
        
    def inference(self, image):
        pixel_values = self.processor(images=image, return_tensors='pt').pixel_values
        generated_ids = self.model.generate(pixel_values, max_length=64)
        generated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        generated_text = unicodedata.normalize("NFC", generated_text)
        
        return generated_text

inferencer = TrOCRInferencer()

def image_to_text(image):
    image = Image.fromarray(image).convert('RGB')
    text = inferencer.inference(image)
    return text

with gr.Blocks() as app:
    gr.Markdown("# Handwritten Image OCR")
    with gr.Tab("Image upload"):
        image = gr.Image(label="Handritten image file")
        output = gr.Textbox(label="Output Box")
        convert_btn = gr.Button("Convert")
        convert_btn.click(
            fn=image_to_text, inputs=image, outputs=output
        )
#         gr.Markdown("## Image Examples")
#         gr.Examples(
#             examples=[
#                 os.path.join(os.getcwd(), "examples/Hello.png"),
#                 os.path.join(os.getcwd(), "examples/Hello_cursive.png"),
#                 os.path.join(os.getcwd(), "examples/Red.png"),
#                 os.path.join(os.getcwd(), "examples/sentence.png"),
#                 os.path.join(os.getcwd(), "examples/i_love_you.png"),
#                 os.path.join(os.getcwd(), "examples/merrychristmas.png"),
#                 os.path.join(os.getcwd(), "examples/Rock.png"),
#                 os.path.join(os.getcwd(), "examples/Bob.png"),
#                 ],
#             inputs=image,
#             outputs=output,
#             fn=image_to_text
#             )
    with gr.Tab("Drawing"):
        gr.Markdown("# Handwritten Image OCR")
        sketchpad = gr.Sketchpad(
            label = "Handwritten Sektchpad",
            shape=(600, 300),
            brush_radius=3,
            invert_colors=False,
            )
        output = gr.Textbox(label="Output Box")
        convert_btn = gr.Button("Convert")
        convert_btn.click(
            fn=image_to_text, inputs=sketchpad, outputs=output
            )
app.launch(inline=False, share=True)

[info] init TrOCR Inferencer


/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://d7820b9b024efa8928.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [1]:
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Initialize the model and tokenizer
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

def translate_ko_to_en(ko_text):
    tokenizer.src_lang = "ko"
    encoded_ko = tokenizer(ko_text, return_tensors="pt")
    generated_tokens = model.generate(**encoded_ko, forced_bos_token_id=tokenizer.get_lang_id("en"))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text

def translate_en_to_ko(en_text):
    tokenizer.src_lang = "en"
    encoded_en = tokenizer(en_text, return_tensors="pt")
    generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("ko"))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text

with gr.Blocks() as app:
    gr.Markdown("# Korean-English and English-Korean Translation Service")
    
    with gr.Tab("Korean to English"):
        ko_text = gr.Textbox(label="Korean Text")
        en_output = gr.Textbox(label="English Translation")
        translate_btn_ko_to_en = gr.Button("Translate")
        translate_btn_ko_to_en.click(fn=translate_ko_to_en, inputs=ko_text, outputs=en_output)
    
    with gr.Tab("English to Korean"):
        en_text = gr.Textbox(label="English Text")
        ko_output = gr.Textbox(label="Korean Translation")
        translate_btn_en_to_ko = gr.Button("Translate")
        translate_btn_en_to_ko.click(fn=translate_en_to_ko, inputs=en_text, outputs=ko_output)

app.launch(inline=False, share=True)


/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `cle

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://ac9cfdc5b89e2c0aa1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
